In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd

In [ ]:
wahlbezirke = pd.read_excel('../data/btw17_erg_Wahlkreis.xlsx')
wahlbezirke = wahlbezirke.fillna(0)

df_wahlkreise = pd.DataFrame()
df_wahlkreise['Wahlbezirk'] = wahlbezirke['Gebiet']
df_wahlkreise['Wahlbezirk_ID'] = wahlbezirke['Nr']

mdb = pd.read_csv('../data/btw17_gewaehlteMDB.csv', sep=";")

colors_parties = {''
    'CDU/CSU': '#000000',
    'SPD': '#ec1b23',
    'AfD': '#08c1ff',
    'FDP': '#fff203',
    'DIE LINKE': '#ff08c1', 
    'BÜNDNIS 90/DIE GRÜNEN': '#71be43'
}

# Regional election results
## Winners in the constituencies

So far we only showed the overall results of first and second votes per party. However, this does not help in understanding regional substructures, therefore in this final part of our contextualizing, we show the strongest party per constituency. In general, this part is intended for the viewer to familiarize with the regional distributions showing the strength and weaknesses of parties in different areas. These can play a role in the important topics during an election campaign and their influence in the voting decision and is thus the missing component for a comprehensive contextualization of our #btw17-dataset. <br>
In the following we present two maps that show every constituency of Germany. The constituencies are colored according to the party who got the most votes in this area. This is done for the first and second vote separately. By hovering over the different regions, the name of the constituency as well as the name of the direct candidate who won there (for the map showing the first votes) or the percentage with which the party won there (for the map showing the second votes) along with some other information can be obtained. <br>
<br>
The geographical information used to create the following maps was taken from the Bundeswahlleiter [5]. 


In [ ]:
parteien_order = ['CDU', 'CSU', 'SPD', 'LINKE', 'GRUENE', 'FDP', 'AfD']
relevant_columns_erst = [p.upper() + '_Erststimmen_Endgültig' for p in parteien_order]
relevant_columns_zweit = [p.upper() + '_Zweitstimmen_Endgültig' for p in parteien_order]

partei_erststimme = []
partei_erg_erststimme = []
partei_zweitstimme = []
partei_erg_zweitstimme = []

for wor in wahlbezirke.iloc:
    stimmen_ergebnisse_erst = [wor[rc] for rc in relevant_columns_erst]
    partei_erststimme.append(parteien_order[np.argmax(stimmen_ergebnisse_erst)])
    partei_erg_erststimme.append(max(stimmen_ergebnisse_erst)/wor['Waehler_Erststimmen_Endgültig'])
    stimmen_ergebnisse_zweit = [wor[rc] for rc in relevant_columns_zweit]
    partei_zweitstimme.append(parteien_order[np.argmax(stimmen_ergebnisse_zweit)])
    partei_erg_zweitstimme.append(max(stimmen_ergebnisse_zweit)/wor['Waehler_Zweitstimmen_Endgültig'])

In [ ]:
df_wahlkreise['Gewinner_Erststimmen_Partei'] = partei_erststimme
df_wahlkreise['Gewinner_Zweitstimmen_Partei'] = partei_zweitstimme
df_wahlkreise['Erg_Erst'] = partei_erg_erststimme
df_wahlkreise['Erg_Zweit'] = partei_erg_zweitstimme

In [ ]:
#add name of the candidate who won the constitutiency
filtered_mdb_erst = mdb[mdb['Gewählt_Stimmenart']=='E']
filtered_mdb_erst = filtered_mdb_erst.sort_values(by=['Gewählt_Wahlkreis_Nr'])
name_mdb_erst = [entry['Vorname'] + ' ' + entry['Name'] for entry in filtered_mdb_erst.iloc]
df_wahlkreise['MdB_Wahlkreis'] = name_mdb_erst

In [ ]:
election_df = df_wahlkreise
geoj = gpd.read_file('../data/Geometrie_Wahlkreise_19DBT_geo.geojson')
geoj = geoj.rename(columns={'WKR_NR':'constitutency number'})

In [ ]:
election_df['Erg_Zweit'] = np.round(election_df['Erg_Zweit']*100, 2)

In [ ]:
#rename columns to better description as hovertext
election_df.rename(columns = {'Gewinner_Erststimmen_Partei':'winning party (first votes)', 'Wahlbezirk': 'constitutency', 'MdB_Wahlkreis':'direct mandate winner', 'Wahlbezirk_ID': 'constitutency number', 'Gewinner_Zweitstimmen_Partei': 'winning party (second votes)', 'Erg_Zweit': 'percentage of votes for winner'}, inplace = True)

In [ ]:
#plotting our election map - first votes
fig = px.choropleth_mapbox(
      election_df,
      geojson = geoj ,
      color = 'winning party (first votes)', #Gewinner_Erststimmen_Partei',
      locations = 'constitutency number',#'Wahlbezirk_ID',
      featureidkey = 'properties.constitutency number',
      center = {'lat': 51.420836, 'lon': 10.373681},
      mapbox_style='white-bg',
      zoom = 5,
      title = 'Election Winners -  First Votes',
      opacity = 1,
      color_discrete_map = {
          'CDU': colors_parties['CDU/CSU'], 'CSU': 'darkgrey', 'SPD': colors_parties['SPD'],
          'FDP': colors_parties['FDP'], 'AfD': colors_parties['AfD'], 'LINKE': colors_parties['DIE LINKE'], 
          'GRUENE': colors_parties['BÜNDNIS 90/DIE GRÜNEN']},
        hover_data = ['constitutency', 'direct mandate winner']
)

fig.update_layout(
    title={ 
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    autosize=False,
    width=800,
    height=800,)

fig.show()

In [ ]:
#plotting our election map - second votes
fig = px.choropleth_mapbox(
    election_df,
    geojson = geoj ,
    color = 'winning party (second votes)',
    locations = 'constitutency number',#'Wahlbezirk_ID',
    featureidkey = 'properties.constitutency number',
    center = {'lat': 51.420836, 'lon': 10.373681},
    mapbox_style='white-bg',#'carto-positron', #white-bg
    zoom = 5,
    title = 'Election Winner - Second Votes',
    opacity = 1,
    color_discrete_map = {
        'CDU': colors_parties['CDU/CSU'], 'CSU': 'darkgrey', 'SPD': colors_parties['SPD'],
        'FDP': colors_parties['FDP'], 'AfD': colors_parties['AfD'], 'LINKE': colors_parties['DIE LINKE'], 
        'GRUENE': colors_parties['BÜNDNIS 90/DIE GRÜNEN'],
    },
    hover_data = ['constitutency', 'percentage of votes for winner']
)

fig.update_layout(
    title={ 
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    autosize=False,
    width=800,
    height=800,)

fig.show()

Both maps show, that in most areas <i>CSU</i> or <i>CDU</i> won. However, in bigger cities this trend is not as clear. Furthermore, in the east of Germany the <i>AfD</i> got a high number of votes and won some constituencies. This can also be said for <i>Die Linke</i>, but in even more constituencies the <i>AfD</i> got the most second votes. Overall, the <i>FDP</i> did not win a constituency or got the most second votes in one of them. The map for the second votes can only be well interpreted in comparison to the map showing the winners of the first votes. This is due to the process in which the second votes (along with the first votes) are used to decide which candidate other than the ones who won a direct mandate become a member of the Bundestag.
Another note has to be made to avoid misleading interpretation: The number of inhabitants per constituency is not equal for all of them. Even though an almost equal number of inhabitants per constituency is aimed at, it is not fully achieved.  